In [1]:
from utils import CNN, SegmentDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

dataset = SegmentDataset("images/", "masks/")
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)
model = CNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [2]:
# Train model
def train(model, dataloader, epochs=5):
	for epoch in range(epochs):
		for img, msk in dataloader:
			outputs = model(img)
			loss = criterion(outputs, msk)

			optimizer.zero_grad()
			loss.backward()
			optimizer.step()
			print(f'epoch: {epoch+1}')

In [3]:
train(model, dataloader)

torch.Size([5, 64, 8, 5])


TypeError: flatten() got an unexpected keyword argument 'dim'